# Pulizia dataset

In [ ]:
#import

import pandas as pd
import numpy as np
import sklearn
import plotly.express as ex

df=pd.read_csv(r"path/to/your/dataset.csv")  # Update with your dataset path

sklearn.set_config(transform_output='pandas')
pd.set_option('display.max_rows', None)         # Nessun limite sul numero di righe stampate
pd.set_option('display.max_colwidth', None)     # Mostra interamente il contenuto delle celle

df=df[df['SEER cause-specific death classification']=='Dead (attributable to this cancer dx)']


In [5]:
df.shape

(714694, 54)

In [58]:
df['Chemotherapy recode (yes, no/unk)'].isna().sum()

np.int64(0)

In [47]:
#rinomina colonne
df.rename(columns={
    'Age recode with <1 year olds and 90+': "age",
    'Sex': "sex",
    'Year of diagnosis': "diagnosis_year",
    'Site recode ICD-O-3 2023 Revision': "site_recode_2023",
    'Primary Site - labeled': "primary_site",
    'Laterality': "laterality",
    'Diagnostic Confirmation': "diagnostic_confirmation",
    'Grade Clinical (2018+)': "clinical_grade",
    'Derived EOD 2018 T Recode (2018+)': "eod_t",
    'Derived EOD 2018 N Recode (2018+)': "eod_n",
    'Derived EOD 2018 M Recode (2018+)': "eod_m",
    'Derived EOD 2018 Stage Group Recode (2018+)': "eod_stage_group",
    'RX Summ--Surg Prim Site (1998+)': "rx_summ_surg_prim_site",
    'RX Summ--Scope Reg LN Sur (2003+)': "rx_summ_scope_reg_ln_sur",
    'RX Summ--Surg Oth Reg/Dis (2003+)': "rx_summ_surg_oth_reg_dis",
    'RX Summ--Surg/Rad Seq': "rx_summ_surg_rad_seq",
    'Reason no cancer-directed surgery': "reason_no_surgery",
    'Radiation recode': "radiation",
    'Chemotherapy recode (yes, no/unk)': "chemo_yes_no",
    'RX Summ--Systemic/Sur Seq (2007+)': "rx_summ_systemic_sur_seq",
    'Time from diagnosis to treatment in days recode': "days_from_diagnosis_to_treatment",
    'EOD Primary Tumor Recode (2018+)': "eod_primary_tumor",
    'EOD Regional Nodes Recode (2018+)': "eod_regional_nodes",
    'EOD Mets Recode (2018+)': "eod_mets",
    'Sentinel Lymph Nodes Examined (2018+)': "n_sentinel_lymph_nodes",
    'Tumor Size Summary (2016+)': "tumor_size_summary",
    'SEER Combined Mets at DX-bone (2010+)': "mets_at_bone",
    'SEER Combined Mets at DX-brain (2010+)': "mets_at_brain",
    'SEER Combined Mets at DX-liver (2010+)': "mets_at_liver",
    'SEER Combined Mets at DX-lung (2010+)': "mets_at_lung",
    'Mets at DX-Distant LN (2016+)': "mets_at_dx_distand_ln",
    'Mets at DX-Other (2016+)': "mets_at_dx_other",
    'Estrogen Receptor Summary (2018+)': "estrogen_receptor",
    'HER2 Overall Summary Recode (2018+)': "her2_summary",
    'Progesterone Receptor Summary (2018+)': "pr_summary",
    'Survival months': "survival_months",
    'First malignant primary indicator': "first_malignant_tumor",
    'Total number of in situ/malignant tumors for patient': 'n_in_situ_malignant_tumors',
    'Total number of benign/borderline tumors for patient': "n_benign_borderline_tumors",
    'Type of Reporting Source': "report_source",
    'Marital status at diagnosis': "marital_status",
    'Median household income inflation adj to 2023': "median_household_income_adj_2023",
    'Rural-Urban Continuum Code': "rural_urban_continuum"
}, inplace=True)



In [48]:
#mappature ed elaborazione valori
df["age"] = df["age"].str.extract(r"(\d+)", expand=False).astype(int) #rendi numerici gli intervalli di età
df.loc[df["days_from_diagnosis_to_treatment"] == '731+ days', "days_from_diagnosis_to_treatment"] = 732
df.loc[df["days_from_diagnosis_to_treatment"] == 'Unable to calculate', "days_from_diagnosis_to_treatment"] = np.nan
cols = ['mets_at_lung', 'mets_at_brain', 'mets_at_bone', 'mets_at_liver']
for col in cols:
    df[col] = df[col].fillna('Not_Applicable')
df['E_R_binary'] = df.apply(
    lambda r: 1 if r['site_recode_2023']=='Breast' and r['estrogen_receptor']=='ER positive' else
              0 if r['site_recode_2023']=='Breast' and r['estrogen_receptor']=='ER negative' else
              np.nan,
    axis=1)

df['her2_binary'] = df.apply(
    lambda r: 1 if r['site_recode_2023']=='Breast' and r['her2_summary']=='HER2 positive' else
              0 if r['site_recode_2023']=='Breast' and r['her2_summary']=='HER2 negative; equivocal' else
              np.nan,
    axis=1)

df['pr_binary'] = df.apply(
    lambda r: 1 if r['site_recode_2023']=='Breast' and r['pr_summary']=='PR positive' else
              0 if r['site_recode_2023']=='Breast' and r['pr_summary']=='PR negative' else
              np.nan,
    axis=1)

mapping = {
    '$120,000+': 120000,
    '$110,000 - $119,999': 115000,
    '$100,000 - $109,999': 105000,
    '$95,000 - $99,999': 97500,
    '$90,000 - $94,999': 92500,
    '$85,000 - $89,999': 87500,
    '$80,000 - $84,999': 82500,
    '$75,000 - $79,999': 77500,
    '$70,000 - $74,999': 72500,
    '$65,000 - $69,999': 67500,
    '$60,000 - $64,999': 62500,
    '$55,000 - $59,999': 57500,
    '$50,000 - $54,999': 52500,
    '$45,000 - $49,999': 47500,
    '$40,000 - $44,999': 42500,
    '< $40,000': 35000,
    'Unknown/missing/no match/Not 1990-2023': None
}

df['median_household_income_adj_2023'] = df['median_household_income_adj_2023'].map(mapping)


label_maps = {
    'sex': {'Male': 0, 'Female': 1},
    'chemo_yes_no': {'No/Unknown': 0, 'Yes': 1},
    'first_malignant_tumor': {'No': 0, 'Yes': 1},
    'Origin recode NHIA (Hispanic, Non-Hisp)': {'Non-Spanish-Hispanic-Latino': 0, 'Spanish-Hispanic-Latino': 1},

}
for col in label_maps:
    df[col] = df[col].map(label_maps[col])

df['days_from_diagnosis_to_treatment'] = pd.to_numeric(df['days_from_diagnosis_to_treatment'], errors='coerce')
df['days_from_diagnosis_to_treatment'] = df['days_from_diagnosis_to_treatment'].fillna(df['days_from_diagnosis_to_treatment'].mean())
df['median_household_income_adj_2023'] = df['median_household_income_adj_2023'].fillna(df['median_household_income_adj_2023'].mean())
df['rx_summ_scope_reg_ln_sur'] = df['rx_summ_scope_reg_ln_sur'].fillna('Unknown or not applicable')


In [51]:
# rimozione di records e colonne
df = df[df['survival_months'] != "Unknown"]
df = df[df['clinical_grade'] != "S"]
df = df[df['Race recode (W, B, AI, API)'] != "Unknown"]
df = df[df['laterality'] != "Only one side - side unspecified"]
df = df[df['diagnostic_confirmation'] != "Unknown"]
df = df[df['rural_urban_continuum'] != "Unknown/missing/no match/Not 1990-2023"]

df.drop(columns=['Site recode ICD-O-3 2023 Revision Expanded'], inplace=True)
df.drop(columns=['Behavior recode for analysis'], inplace=True)# tutti i tumori nel dataset sono maligni
df.drop(columns=['Race recode (W, B, AI, API)'], inplace=True)
df.drop(columns=['laterality'], inplace=True)
df.drop(columns=['AJCC ID (2018+)'], inplace=True)

In [54]:
# salvataggio
df.to_csv( r'C:\ML _SUPERVISED\progetto esame\df_clean_5.csv')

# esplorazione variabili numeriche

In [ ]:
# grafici per visualizzare relazioni tra survival_months e variabili numeriche
df=pd.read_csv(r'C:\ML _SUPERVISED\progetto esame\df_clean_3.csv')
import plotly.express as px

columns = [
    'age',
    'days_from_diagnosis_to_treatment',
    'n_in_situ_malignant_tumors',
    'n_benign_borderline_tumors',
    'median_household_income_adj_2023'
]

for col in columns:
    fig = px.scatter(
        df,
        x=col,
        y='survival_months',
        title=f'{col} vs. survival_months',
        #opacity=0.6
    )
    fig.show()
